SQL Introduction
---
Besides the powerful python packages for database, pandas, polars, et.al,  most developers are familiar with SQL (Structured Query Language), which  is one of the easiest and most natural ways for developers to retrieve data and it is well-supported in Python. Here we introduce Duckdb", which is designed to support Online Analytical Processing (OLAP), i.e. we can utilize it online without complicated installation!

Outlines
---
1. make the data,
2. load the data as SQL 
3. example of SQL with query
4. artifact maker

```
Chapgpt Prompt (you can try by yourself) 

1. col1 for single selection of kinds
2. col2 for multiple selection of Countries
```

In [1]:
# import packages
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go

import pandas as pd

%matplotlib inline

In [2]:
# Get the data

countries=['US','AU','CA','DE','FR','GB','IE','JP']

C=["https://raw.githubusercontent.com/hiring-lab/job_postings_tracker/master/{0}/aggregate_job_postings_{0}.csv".format(country,country) for country in countries]


In [3]:
# import  data  to Database
dfs=[pd.read_csv(c,parse_dates=['date']) for c in C]

df_world=pd.DataFrame()
df_world['date']=dfs[0]['date']
df_world['variable']=dfs[0]['variable']
for i in range(len(countries)):
    df_world[countries[i]]=dfs[i]['indeed_job_postings_index_SA']

df_world.reset_index(inplace=True,drop=True)

In [4]:
df_world

,date,variable,US,AU,CA,DE,FR,GB,IE,JP
0,2020-02-01,total postings,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
1,2020-02-02,total postings,99.98,100.06,100.08,100.01,99.95,99.95,99.91,99.64
2,2020-02-03,total postings,99.97,100.22,100.12,100.13,100.03,100.01,99.83,99.54
3,2020-02-04,total postings,100.03,100.36,100.18,99.71,99.86,99.95,99.67,99.15
4,2020-02-05,total postings,100.12,100.39,100.30,99.91,99.84,99.98,99.62,98.89
...,...,...,...,...,...,...,...,...,...,...
2305,2023-03-27,new postings,146.79,195.99,134.37,172.77,161.16,125.70,168.13,175.26
2306,2023-03-28,new postings,147.48,196.62,134.28,174.77,161.87,125.81,169.43,174.30
2307,2023-03-29,new postings,147.79,196.34,134.44,176.32,162.79,126.31,170.42,174.64
2308,2023-03-30,new postings,147.88,197.03,134.60,178.71,163.20,127.00,171.46,176.68


In [12]:
# packages for SQL
import sqlite3
import sqlalchemy as sa

In [13]:
# Create a connection to your SQL database 
# saved df_world to a file "Job.db" 
engine = sa.create_engine('sqlite:///Job.db')

# Write the dataframe to a new SQL table named 'job'
df_world.to_sql('job', con=engine, if_exists='replace', index=False)

2310

In [16]:
# Load SQL, 透過SQL語法讀取資料庫中的資料
conn = sqlite3.connect('Job.db') 
SQL_df = pd.read_sql("SELECT * FROM job WHERE variable='new postings'", conn, parse_dates=['date'])
SQL_df

,date,variable,US,AU,CA,DE,FR,GB,IE,JP
0,2020-02-01,new postings,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
1,2020-02-02,new postings,100.86,97.80,100.17,100.92,100.02,100.24,100.21,100.43
2,2020-02-03,new postings,101.71,97.92,100.37,101.70,100.11,100.43,100.39,102.56
3,2020-02-04,new postings,101.99,98.95,100.58,100.93,99.03,100.02,100.02,101.64
4,2020-02-05,new postings,102.21,99.14,100.88,101.74,98.75,99.91,100.28,101.54
...,...,...,...,...,...,...,...,...,...,...
1150,2023-03-27,new postings,146.79,195.99,134.37,172.77,161.16,125.70,168.13,175.26
1151,2023-03-28,new postings,147.48,196.62,134.28,174.77,161.87,125.81,169.43,174.30
1152,2023-03-29,new postings,147.79,196.34,134.44,176.32,162.79,126.31,170.42,174.64
1153,2023-03-30,new postings,147.88,197.03,134.60,178.71,163.20,127.00,171.46,176.68


In [17]:
SQL_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      1155 non-null   datetime64[ns]
 1   variable  1155 non-null   object        
 2   US        1155 non-null   float64       
 3   AU        1155 non-null   float64       
 4   CA        1155 non-null   float64       
 5   DE        1155 non-null   float64       
 6   FR        1155 non-null   float64       
 7   GB        1155 non-null   float64       
 8   IE        1155 non-null   float64       
 9   JP        1155 non-null   float64       
dtypes: datetime64[ns](1), float64(8), object(1)
memory usage: 90.4+ KB


Duckdb
---

In [14]:
import duckdb

con = duckdb.connect(database='Job.db', read_only=True) 

In [15]:
# list the first 5 data
data_query = "FROM 'job' LIMIT 5"
con.execute(data_query).df()

,date,variable,US,AU,CA,DE,FR,GB,IE,JP
0,2020-02-01,total postings,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
1,2020-02-02,total postings,99.98,100.06,100.08,100.01,99.95,99.95,99.91,99.64
2,2020-02-03,total postings,99.97,100.22,100.12,100.13,100.03,100.01,99.83,99.54
3,2020-02-04,total postings,100.03,100.36,100.18,99.71,99.86,99.95,99.67,99.15
4,2020-02-05,total postings,100.12,100.39,100.30,99.91,99.84,99.98,99.62,98.89


In [16]:
# query for distinct values in variable and display ordered
query="""
SELECT 
     DISTINCT variable
From job        
ORDER BY variable       
"""

con.execute(query).df()

,variable
0,new postings
1,total postings


In [17]:
# query for all (date, US) paired data and variable=new postings
query_1="""
SELECT 
        date,US 
From job        
WHERE variable =  'new postings'          
"""

con.execute(query_1).df()

,date,US
0,2020-02-01,100.00
1,2020-02-02,100.86
2,2020-02-03,101.71
3,2020-02-04,101.99
4,2020-02-05,102.21
...,...,...
1150,2023-03-27,146.79
1151,2023-03-28,147.48
1152,2023-03-29,147.79
1153,2023-03-30,147.88


Self-Practicing
---
Query all the data pairs (data, US,JP) with variable="total postings"

Streamlit Artifact
---

In [19]:
import streamlit as st
import altair as alt

In [20]:
# query for all data
query="""
   SELECT * 
   FROM Job
"""
#date=list(con.execute(query).df().columns)[0]
#kinds=list(con.execute(query).df().columns)[1]
Countries=list(con.execute(query).df().columns)[2:]

In [22]:
con.execute(query).df()

,date,variable,US,AU,CA,DE,FR,GB,IE,JP
0,2020-02-01,total postings,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
1,2020-02-02,total postings,99.98,100.06,100.08,100.01,99.95,99.95,99.91,99.64
2,2020-02-03,total postings,99.97,100.22,100.12,100.13,100.03,100.01,99.83,99.54
3,2020-02-04,total postings,100.03,100.36,100.18,99.71,99.86,99.95,99.67,99.15
4,2020-02-05,total postings,100.12,100.39,100.30,99.91,99.84,99.98,99.62,98.89
...,...,...,...,...,...,...,...,...,...,...
2305,2023-03-27,new postings,146.79,195.99,134.37,172.77,161.16,125.70,168.13,175.26
2306,2023-03-28,new postings,147.48,196.62,134.28,174.77,161.87,125.81,169.43,174.30
2307,2023-03-29,new postings,147.79,196.34,134.44,176.32,162.79,126.31,170.42,174.64
2308,2023-03-30,new postings,147.88,197.03,134.60,178.71,163.20,127.00,171.46,176.68


Notice
---
Streamlit artifact is availed for web-service; this means it would only work while the server is up and worked without error. If you have install a completed python environment, (for instance, Anaconda Jupyter, VS code + Python", you can activate streamlit artifact, app.py, in console:
```shell
streamlit run app.py
```
and test by the IP address: 
```
localhost:8501
```

Code Explanation
---
Two single-select options, one for "variable", and the other for "country".

In [8]:
st.subheader('Filters')

col1, col2 = st.columns(2)

with col1:
    query="""
            SELECT 
                 DISTINCT variable
            From job        
            ORDER BY variable       
          """

    kinds=con.execute(query).df()
    kind = st.selectbox('Kind of Statistics',kinds)
with col2: 
    country = st.selectbox('Country',Countries)


  command:

    streamlit run /Users/chu-chinghuang/opt/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py [ARGUMENTS]
2023-04-07 22:23:51.382 
  command:

    streamlit run /Users/chu-chinghuang/opt/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py [ARGUMENTS]


Full Code, app.py
---
```
import pandas as pd
import streamlit as st

import altair as alt
import duckdb

con = duckdb.connect(database='Job.db', read_only=True) 

# Coutries
query="""
   SELECT * 
   FROM job
"""
Countries=list(con.execute(query).df().columns)[2:]


st.subheader('Investingation')

col1, col2 = st.columns(2)

with col1:
    query="""
            SELECT 
                 DISTINCT variable
            From job        
            ORDER BY variable       
          """

    kinds=con.execute(query).df()
    kind = st.selectbox('Kind of Statistics',kinds)
with col2: 
    country = st.selectbox('Country',Countries)
    

result_df = con.execute("""
    SELECT 
        *
    FROM Job 
    WHERE variable=?
    """, [kind]).df()

chart = alt.Chart(result_df).mark_circle().encode(
    x = 'date',
    y = country,
    #color = 'carrier'
).interactive()
st.altair_chart(chart, theme="streamlit", use_container_width=True)
    
```

Finally
---
1. create a folder, named "streamlit-SQL"
2. copy `app.py, Job.db, into this folder
3. also create a file, requirements.txt, which includes packages required in artifact:
```
dockdb
altair
```
4. create new github and push files into repo;
5. sign in streamlit official to create artifact. 

[Streamlit Airtifact](https://cchuang2009-streamlit-sql-app-68vvn2.streamlit.app/)

In [ ]:
!pip install duckdb-engine sqlalchemy

In [11]:
engine = sa.create_engine('duckdb:///Job.duckdb')

# Write the dataframe to a new SQL table named 'job'
df_world.to_sql('job', con=engine, if_exists='replace', index=False)

-1

In [17]:
con = duckdb.connect(database='Job.duckdb')
#preview_data_query = "SELECT COUNT(*) FROM 'itineraries_snappy.parquet'"
#con.execute(preview_data_query).df()
data_df = pd.read_sql("SELECT * FROM job where variable='total postings'", con,parse_dates=['date'])
data_df

<ipython-input-17-d883c91b8c54>:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_df = pd.read_sql("SELECT * FROM job where variable='total postings'", con,parse_dates=['date'])


,date,variable,US,AU,CA,DE,FR,GB,IE,JP
0,2020-02-01,total postings,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
1,2020-02-02,total postings,99.98,100.06,100.08,100.01,99.95,99.95,99.91,99.64
2,2020-02-03,total postings,99.97,100.22,100.12,100.13,100.03,100.01,99.83,99.54
3,2020-02-04,total postings,100.03,100.36,100.18,99.71,99.86,99.95,99.67,99.15
4,2020-02-05,total postings,100.12,100.39,100.30,99.91,99.84,99.98,99.62,98.89
...,...,...,...,...,...,...,...,...,...,...
1150,2023-03-27,total postings,134.10,208.37,144.05,147.28,164.48,125.68,159.34,126.96
1151,2023-03-28,total postings,133.98,209.06,143.87,147.42,164.27,125.46,159.21,126.46
1152,2023-03-29,total postings,133.88,209.37,143.79,147.55,164.61,125.39,158.93,126.05
1153,2023-03-30,total postings,133.93,209.84,143.95,147.71,164.66,125.38,158.61,125.64
